# **SQL 102 - Advanced Querying Techniques**
Author: Martin Arroyo

### **About this notebook**

All of your queries will be written using preloaded databases that are available only in this notebook. Our "RDBMS" and SQL dialect is called `duckdb`, a new and popular Python library that provides the framework to make our queries possible. You can find [the documentation for `duckdb` here](https://duckdb.org/docs/sql/introduction) - you will want to keep the documentation handy.

`teachdb`, which provides the data that you will be working with, is a Python library written by The Freestack Initiative, a group of COOP alumni who want to empower the community to learn and improve their technical skills by providing materials and resources at low (or no) cost.

## **How to use this notebook**

First, we'll do a quick tutorial on how to use the notebook with these tools, then we'll dive into more SQL!

### **Step 1: Press the play button below to set up the database and notebook**

You will see a checkmark appear when the database is finished setting up.

In [ ]:
%%capture --no-stderr
# @title Press Play { display-mode: "form" }

# This code is used to set up the notebook by installing the libraries we need, configuring extensions to
# make displays for our queries look nice, and connecting to our relational database so that you can write
# queries in code cells using the %%sql magic tag.

# Install `teachdb` if it's not in the system already
%pip install --quiet --upgrade git+https://github.com/freestackinitiative/teachingdb.git
from teachdb.teachdb import connect_teachdb
# Set configurations for notebook & Load data
con = connect_teachdb(database=["core", "restaurant"])
%sql con

# Check out the Freestack Initiative @https://github.com/freestackinitiative

### **Step 2: Run a query**

To run SQL queries against the database, create a new code cell. Then write `%%sql` at the top. This tells the notebook that this cell is being used to query the database. You can write your queries underneath the `%%sql` line by pressing the play button of the cell or selecting the cell and using `CTRL + Shift + Enter` on Windows (`CMD + Shift + Return` on Mac.)

Go ahead and try it by executing the query in the cell below:

In [ ]:
%%sql

SELECT *
FROM Customers
LIMIT 5

## **Single Table Query Review and `CASE` Statements**

In this section, we'll warm up with some review of the basics we learned in SQL 101. Then you'll learn a new query technique - `CASE` statements - which are a special kind of conditional statement that lets us create custom column values based on conditions we specify.

### **Review - Single Table Queries and Aggregation**

Let's get warmed up by writing a query using what we learned in SQL 101!

Write a query that shows the top 5 customers in the `Reservations` table that have the most reservations. Additionally, show the average party size for each of those customers.

`Expected Output:`

| CustomerID | TotalReservations | AvgPartySize |
|------------|-------------------|--------------|
| 6          | 34                | 3.882353     |
| 80         | 30                | 4.033333     |
| 31         | 27                | 3.259259     |
| 41         | 27                | 3.962963     |
| 44         | 27                | 3.777778     |

<br/>
<details>
<summary>Click here to reveal answer</summary>

```sql
SELECT CustomerID, COUNT(ReservationID) AS total_reservations, AVG(PartySize) AS avg_party_size
FROM Reservations
GROUP BY CustomerID
ORDER BY total_reservations DESC
LIMIT 5
```

</details>

In [19]:
%%sql

SELECT CustomerID, COUNT(ReservationID) AS TotalReservations, ROUND(AVG(PartySize),2) AS AVGPartySize
FROM Reservations
GROUP BY CustomerID
ORDER BY TotalReservations DESC
LIMIT 5;



,CustomerID,TotalReservations,AVGPartySize
0,6,34,3.88
1,80,30,4.03
2,41,27,3.96
3,47,27,4.26
4,44,27,3.78


Great work with that first query! Now that we're warmed up, it's time to take your querying skills to the next level!

## **`CASE` Statements**

A popular restaurant reviewer is using a pricing scale that they invented to rate how affordable dishes at restaurants are. Restaurants that have too many dishes considered "Pricey" typically have lower ratings, while those with more dishes in the "Average" range do best. We don't want our restaurant to have a low rating, so we have to find out how our menu does on this scale.

Here is the reviewer's pricing scale:

>\$4 or less - `Inexpensive`
>
>Between \$4 and \$8 - `Average`
>
>Above \$8 - `Pricey`

How can we convert the price of a dish to one of these three values based on the price? This is a perfect use for `CASE` statements!


#### **How to use `CASE` statements**

`CASE` statements are very similar to using the `IF` function in Excel. They allow you to specify "If/Then/Else" logic in your queries.

Here is the general form of a `CASE` statement:

```sql
CASE WHEN {`some condition to check`} THEN {`value if the condition is true`} ELSE {`value if all other conditions are false`} END
```

If you need to check more than one separate condition, you simply add another `WHEN/THEN` clause. The `ELSE` is always the final condition checked since it covers the case where all the other cases are false.

To see this in action, check out the query we use to show the pricing scale for our menu.

#### **`CASE` Statement Example**

Here is the query to check the pricing scale:
```sql
SELECT Name
    , Price
    , Type
    , CASE
        WHEN Price <= 4.0 THEN 'Inexpensive' -- Check the first condition
        WHEN Price BETWEEN 4.0 AND 8.0 THEN 'Average' -- Check the second condition
        ELSE 'Pricey' -- Anything over 8.0 is Pricey
      END AS PriceRating -- We end our CASE statement and give the resulting column a name using an alias
FROM Dishes
```

Here are the first five results:

| Name                         | Price | Type      | PriceRating |
|------------------------------|-------|-----------|-------------|
| Parmesan Deviled Eggs        | 8.00  | Appetizer | Average     |
| Artichokes with Garlic Aioli | 9.00  | Appetizer | Pricey      |
| French Onion Soup            | 7.00  | Main      | Average     |
| Mini Cheeseburgers           | 8.00  | Main      | Average     |
| Panko Stuffed Mushrooms      | 7.00  | Appetizer | Average     |

> **Extra Context:** <em>Code Comments</em>
>
> You may have noticed that we used "`--`" followed by some text in the query above. These lines of text are called "comments", which are notes in the code written by the developer to
communicate what a particular line or section of code means. All coding languages have a way for developers to leave these comments in their code so that they can
let future readers understand their thought process and why something was done.
>
> We use a comment symbol to let the computer know that "this is not code, so don't execute it!" Each programming language will have its own symbol for comments. In SQL, it is most common
to use "`--`" for a single-line comment. Check the documentation for the SQL dialect you are using to be sure, though!

#### Breakdown - `CASE` Statement query

Our query looks pretty simple aside from the `CASE` statement that we added. We'll focus on breaking that down line-by-line:

```sql
    CASE WHEN Price <= 4.0 THEN 'Inexpensive'
```

Every `CASE` statement begins with the word `CASE`. After that, we check conditions using the `WHEN`/`THEN` keywords. The `WHEN` looks at the condition that you specify and the `THEN` defines what happens when that condition is true. In this case, we're checking if the price is $4 or less; if it is, we're telling SQL that the value we want is "Inexpensive".

```sql
    WHEN Price BETWEEN 4.0 AND 8.0 THEN 'Average'
```

Since we need to check more than one condition to look at the price scale, we need to add another `WHEN`/`THEN` set of keywords to check the next condition. This one says, <em>"Use the word `Average` here if the `Price` is between $4 and $8."</em>

```sql
    ELSE 'Pricey'
```

The `ELSE` portion of the `CASE` statement determines what to do if all of the conditions before it are false. This is saying, <em>"If the price of a dish is not `Inexpensive` or `Average`, then it is `Pricey`"</em>

```sql
    END AS PriceRating
```

We use the `END` keyword to close all `CASE` statements. This says, <em>"We are done with our statement."</em> Since we are creating a column that doesn't otherwise exist in the database, the RDBMS will give it a default name. It is best practice to name our `CASE` statements using aliases. Here, we name the resulting column from our `CASE` statement `PriceRating`.

#### **Action Item - Using the `CASE` Statement**

The restaurant reviewer recently published an update to their price scale! The new scale is below:

> **Restaurant Reviewer's NEW Pricing Scale:**
>
>\$3 or less - `Super Cheap`
>
>Between \$4 and \$5 - `Inexpensive`
>
>Between \$5 and \$8 - `Average`
>
>Above \$8 - `Pricey`

Write a query using the `Dishes` table that shows the updated price scale. Your results should include the name of the dish, the price, and it's type - along with the new `PriceRating`. Order your results by `Price` so that we see the lowest prices first and limit your results to just the first five rows.

`Expected Output:`

| Name                   | Price | Type     | PriceRating |
|------------------------|-------|----------|-------------|
| Pomegranate Iced Tea   | 4.00  | Beverage | Inexpensive |
| Apple Pie              | 5.00  | Dessert  | Inexpensive |
| Chocolate Chip Brownie | 6.00  | Dessert  | Average     |
| Tropical Blue Smoothie | 6.00  | Beverage | Average     |
| Cafe Latte             | 6.00  | Beverage | Average     |

<br/>
<details>
    <summary>Click here to reveal the answer</summary>

```sql
SELECT Name
    , Price
    , Type
    , CASE
        WHEN Price <= 3.0 THEN 'Super Cheap'
        WHEN Price BETWEEN 4.0 AND 5.0 THEN 'Inexpensive'
        WHEN Price BETWEEN 5.0 AND 8.0 THEN 'Average'
        ELSE 'Pricey' -- Anything over 8.0 is Pricey
    END AS PriceRating
FROM Dishes
ORDER BY Price
LIMIT 5
```

</details>

In [35]:
%%sql

SELECT Name, Price, Type, CASE
  WHEN Price <= 3 THEN 'Super Cheap'
  WHEN Price BETWEEN 4 AND 5 THEN 'Inexpensive'
  WHEN Price BETWEEN 5 AND 8 THEN 'Average'
  ELSE 'Pricey'
END AS PriceRating
FROM Dishes
ORDER BY Price ASC
LIMIT 5;

,Name,Price,Type,PriceRating
0,Pomegranate Iced Tea,4.0,Beverage,Inexpensive
1,Apple Pie,5.0,Dessert,Inexpensive
2,Chocolate Chip Brownie,6.0,Dessert,Average
3,Tropical Blue Smoothie,6.0,Beverage,Average
4,Cafe Latte,6.0,Beverage,Average


## **Scalar Functions**

So far, we have primarily used aggregate functions (e.g. `COUNT`, `SUM`) in our queries, which collapse our results into a single row. However, there is another major type of function that you should be awawre of called **scalar functions**. Scalar functions operate on individual values and return a single value. They are essential for performing common operations such as string manipulation, date handling, and mathematical calculations. Understanding scalar functions in SQL will enhance your ability to manipulate and transform data effectively.

By the end of this section, you'll understand:

- What scalar functions are
- Why you'd use them
- How to incorporate them into your queries

### **What are Scalar Functions?**

Imagine you're a chef who needs to adjust recipes based on the number of servings. Scalar functions in SQL are like the kitchen tools that help you perform these adjustments—whether it's slicing ingredients to a specific size or calculating the cooking time based on the recipe.

In SQL, scalar functions take one or more inputs and return a single output. They are used to transform or calculate values at the row level.

### **Why Use Scalar Functions?**

Scalar functions are incredibly useful when:

- You need to format or manipulate string values.
- You want to perform calculations on numeric data.
- You need to handle and manipulate date and time values.

> #### ☝️ **A note about using scalar functions**
>
> If you recall from the Database Crash Course, there are many dialects of SQL that each have their own implementation. This means that not every SQL database will have the same functions available. This means that whenever you want to use a scalar function in your query, its important to double-check the documentation for the database you are using to see which functions are available to use.
>
>The SQL dialect that we are using for this class is `duckdb`, and the [documentation is available here](https://duckdb.org/docs/sql/functions/overview). You can review the available functions there.
>
> Also important: Scalar functions can be used in `SELECT`, `WHERE`, and `GROUP BY` clauses - they are not limited to your select statements only, so you can aggregate and/or filter on the results of a function.

### **Example - Using Scalar Functions to work with dates**

One very common use for scalar functions is to manipulate dates in our tables. Typically, we'll want to convert a timestamp to a date (or vice versa) or extract some information from a date like the day of the week, year, month, etc.

First, let's show you how to convert a `TIMESTAMP` (e.g. `2022-12-01 11:59:30`) to its `DATE` representation (e.g. `2022-12-01`) using the [`CAST` function](https://duckdb.org/docs/sql/expressions/cast.html). We'll use the `Reservations` table for this exercise.

Here's the query:

```sql
SELECT
    ReservationID,
    "Date" AS ReservationTimestamp,
    CAST("Date" AS DATE) AS ReservationDate,
    PartySize
FROM Reservations
LIMIT 5
```

And here are the results:

| ReservationID | ReservationTimestamp | ReservationDate | PartySize |
|---------------|----------------------|-----------------|-----------|
| 1             | 2018-06-01 15:30:00  | 2018-06-01      | 6         |
| 2             | 2018-06-02 13:30:00  | 2018-06-02      | 2         |
| 3             | 2018-06-04 08:00:00  | 2018-06-04      | 4         |
| 4             | 2018-06-04 19:30:00  | 2018-06-04      | 5         |
| 5             | 2018-06-06 13:00:00  | 2018-06-06      | 1         |


### **Breakdown**

The `CAST` function converts a column from one data type to another. The way the syntax works is like this:

`CAST([Column to Convert] AS [Datatype we want to convert to])`

So in our case, we want to convert the data in the "Date" column from a `TIMESTAMP` to a regular `DATE` and remove the `hour:min:sec` part. That's why we use:

```sql
SELECT CAST("Date" AS DATE) AS ReservationDate
FROM Reservations
```

We're telling the function to ***"Convert the `Date` column from a `TIMESTAMP` to a `DATE` data type"***.

Why would we want to do this? Well, there can be numerous reasons depending on what kinds of questions you want to answer with your data. A common use for this is to get a list of unique dates. This would be difficult with the `TIMESTAMP` because there can be multiple unique entries for a single date. Converting the column to a `DATE` removes the time component and lets you find distinct dates.

### **Example - Extracting parts of a date**

Another very common use case for date functions is extracting parts out of a given date. For example, let's say that we want to extract the year from our date. We could do this:

```sql
SELECT DISTINCT YEAR("Date") AS ReservationYear
FROM Reservations
ORDER BY ReservationYear
```

And we will get all of the years that are on record in the `Reservations` table, in order:

| ReservationYear |
|-----------------|
| 2018            |
| 2019            |
| 2020            |
| 2021            |
| 2022            |

If we wanted to extract just the month from our date, we could use the `MONTH` function to do that:

```sql
SELECT DISTINCT MONTH("Date") AS ReservationMonth
FROM Reservations
ORDER BY ReservationMonth
```

| ReservationMonth |
|------------------|
| 1                |
| 2                |
| 3                |
| 4                |
| 5                |
| 6                |
| 7                |
| 8                |
| 9                |
| 10               |
| 11               |
| 12               |

#### The `EXTRACT` function

While those functions work well for extracting some parts of a date, there is a more general and powerful function that can be used - `EXTRACT`. `EXTRACT` works by having a list of [known subfields](https://duckdb.org/docs/sql/functions/datepart) from a `DATE` or `TIMESTAMP` that it can extract, and the syntax is like this:

`EXTRACT([subfield] FROM [date column])`

Here's an example query:

```sql
SELECT
    EXTRACT('year' FROM "Date") AS ReservationYear,
    EXTRACT('month' FROM "Date") AS ReservationMonth
FROM
    Reservations
WHERE
    EXTRACT('year' FROM "Date") = 2020 -- We can use functions in both WHERE and GROUP BY clauses
ORDER BY
    ReservationYear,
    ReservationMonth
```

And the results:

| ReservationYear | ReservationMonth |
|-----------------|------------------|
| 2020            | 1                |
| 2020            | 2                |
| 2020            | 3                |
| 2020            | 4                |
| 2020            | 5                |
| 2020            | 6                |
| 2020            | 7                |
| 2020            | 8                |
| 2020            | 9                |
| 2020            | 10               |
| 2020            | 11               |
| 2020            | 12               |


### **Action Item - Extracting the Day of the Week from a `TIMESTAMP`**

**Scenario**

The restaurant would like to start keeping track of which days of the week get the most reservations. But the current `Reservations` table doesn't list the day of the week - it only has the `TIMESTAMP` for the reservation itself. Write a query that extracts the day of the week from the `Date` column and put it in a column called `DayOfWeek`.

`Expected Output:`

| ReservationID | Date                | DayOfWeek | PartySize |
|---------------|---------------------|-----------|-----------|
| 1             | 2018-06-01 15:30:00 | Friday    | 6         |
| 2             | 2018-06-02 13:30:00 | Saturday  | 2         |
| 3             | 2018-06-04 08:00:00 | Monday    | 4         |
| 4             | 2018-06-04 19:30:00 | Monday    | 5         |
| 5             | 2018-06-06 13:00:00 | Wednesday | 1         |

<br/>
<details>
<summary>Here's a hint in case you're stuck</summary>

The `dayofweek` subfield returns the day of the week as a number (Sunday = 0, Saturday = 6), but we want the actual name. Can we leverage the `CASE` statement to help us get the name of the day?
</details>
<br/>
<details>
<summary>Click here to reveal answer</summary>

```sql
SELECT
    ReservationID,
    "Date",
    CASE EXTRACT('dayofweek' FROM "Date")
        WHEN 0 THEN 'Sunday'
        WHEN 1 THEN 'Monday'
        WHEN 2 THEN 'Tuesday'
        WHEN 3 THEN 'Wednesday'
        WHEN 4 THEN 'Thursday'
        WHEN 5 THEN 'Friday'
        WHEN 6 THEN 'Saturday'
    END AS DayOfWeek,
    PartySize
FROM
    Reservations
ORDER BY
    ReservationID,
    "Date"
```

</details>

In [39]:
%%sql

SELECT

,ReservationYear,ReservationMonth
0,2020,1
1,2020,1
2,2020,1
3,2020,1
4,2020,1
5,2020,1
6,2020,1
7,2020,1
8,2020,1
9,2020,1


## **Subqueries - Writing a query within a query**

You've come a long way in your SQL journey! You're already familiar with the basics like `SELECT`, `WHERE`, and `GROUP BY`. And you just learned how to add conditional logic using the `CASE` statement. Now, let's take it up a notch by diving into the concept of subqueries.

In SQL, a subquery is essentially a query within a query. This might sound a little confusing at first, but don't worry - by the end of this section, you'll understand:

- What subqueries are
- Why you'd use them, and
- How to incorporate them into your SQL work

### **What is a Subquery?**

Imagine you're a detective. You've got a briefcase full of clues, and inside that briefcase is a smaller box with even more specific clues. A subquery works like that smaller box of clues, providing additional information to the main query (your detective's briefcase).

In SQL, a subquery is a query nested inside another query. It's used to retrieve data that will be utilized by the main query to further filter or refine the results. The subquery is executed first, and its result is passed to the main query.

### **Why Use Subqueries?**

Subqueries can be incredibly useful when:
- You need to filter rows based on values from another query.
- You require multiple logical steps in a single query.

### **Example - Using Basic Subqueries as a filter**

**Scenario:**

Let's say we want to find the names of the customers who ordered dishes most recently from us. Customer dish orders are found in the `CustomersDishes` table, but it doesn't have the customer names. The customer names are found in the `Customers` table.

We could do a join, but there is another way that doesn't involve one - using a subquery!

**Breakdown:**

First, what we want to do is create a query that gets us a list of the CustomerID's in `CustomersDishes`. This is going to be our **subquery**:

```sql
SELECT DISTINCT CustomerID
FROM CustomersDishes
```

We use the results from the **subquery** to feed data to the "main" query for processing.

**The full subquery:**

```sql
-- This outer section is the main or "outer" query
SELECT FirstName, LastName
FROM Customers
-- This is where we check the subquery for the customer ids
WHERE CustomerID IN (SELECT DISTINCT CustomerID FROM CustomersDishes)
```

**How it works:**

The main/outer query gets the first and last names in the `Customers` table. Then, in our `WHERE` clause, we check for customer IDs found in our subquery result `(SELECT DISTINCT CustomerID FROM CustomersDishes)`.

> **Important:** Subqueries are ALWAYS surrounded by parentheses

The outer query will filter the results for the `Customers` table only using the customer IDs found in the `CustomersDishes` table.

**Caveat**

When using a subquery as a filter (like the previous example), the subquery must only return a single column - otherwise you will get an error.

However, it is possible to also use a subquery to return results like we were querying another table. We will show an example of that shortly

#### **Action Item - Write a basic subquery as a filter**

Write a query that finds the FirstName and LastName of customers who have an event scheduled in the `CustomersEvents` table.

`Expected Output:`

| FirstName | LastName    |
|-----------|-------------|
| Taylor    | Jenkins     |
| Cammi     | Kynett      |
| Cleo      | Goldwater   |
| Winnah    | D'Elia      |
| Ab        | Jurn        |
| Lelah     | Seathwright |

<br/>
<details>
<summary>Click here to reveal answer</summary>

```sql
SELECT FirstName, LastName
FROM Customers
WHERE CustomerID IN (SELECT DISTINCT CustomerID FROM CustomersEvents)
```

</details>


In [ ]:
%%sql


---

### **Lab 1 - Subqueries to answer multi-step questions**

As mentioned earlier, subqueries are great for answering questions that require multiple steps to answer. The basic strategy is to write a subquery that completes one step, typically creating a table that can then be used in the next step. We'll work through an example together to help solidify this concept.

Let's return to our earlier example with the restaurant reviewers price scale.

**Scenario:**

This time, we want to **find how many dishes we have that are in each category of the price scale**. Logically, this requires two steps to answer:

1. Create a table that shows the pricing scale
2. Write a query against that table that counts the number of dishes for each category

First, let's look at the query we'll use to do this and then break it down step-by-step:

```sql
-- This is our main query
SELECT PriceRating, COUNT(*) AS TotalDishes
FROM
-- START OF SUBQUERY
(
    SELECT Name
    , Price
    , Type
    , CASE
        WHEN Price <= 3.0 THEN 'Super Cheap'
        WHEN Price BETWEEN 4.0 AND 5.0 THEN 'Inexpensive'
        WHEN Price BETWEEN 5.0 AND 8.0 THEN 'Average'
        ELSE 'Pricey'
    END AS PriceRating
FROM Dishes) AS PriceRatingTable
-- END OF SUBQUERY
GROUP BY PriceRating
ORDER BY TotalDishes DESC
```

Run this query in the cell below to see how it all works together:

In [38]:
%%sql
SELECT PriceRating, COUNT(*) AS TotalDishes
FROM
-- START OF SUBQUERY
(
    SELECT Name
    , Price
    , Type
    , CASE
        WHEN Price <= 3.0 THEN 'Super Cheap'
        WHEN Price BETWEEN 4.0 AND 5.0 THEN 'Inexpensive'
        WHEN Price BETWEEN 5.0 AND 8.0 THEN 'Average'
        ELSE 'Pricey'
    END AS PriceRating
FROM Dishes) AS PriceRatingTable
-- END OF SUBQUERY
GROUP BY PriceRating
ORDER BY TotalDishes DESC

,PriceRating,TotalDishes
0,Pricey,11
1,Average,9
2,Inexpensive,2


#### **Breakdown - Subqueries to answer multi-step questions**

There's quite a bit happening in that query, and it might look confusing, too. But let's break it down piece-by-piece:

**Let's start with the subquery:**

```sql
(
    SELECT Name
    , Price
    , Type
    , CASE
        WHEN Price <= 3.0 THEN 'Super Cheap'
        WHEN Price BETWEEN 4.0 AND 5.0 THEN 'Inexpensive'
        WHEN Price BETWEEN 5.0 AND 8.0 THEN 'Average'
        ELSE 'Pricey'
    END AS PriceRating
FROM Dishes) AS PriceRatingTable
```

This subquery is helping us complete `step 1` of our query. It's creating the `PriceRatingRable` that gives a price rating to all of our dishes. The reason we are creating this as a subquery is because:

1. The table does not currently exist in the database
2. We may not have permission or otherwise be able to create a new table in a database. In these cases, we can use subqueries to create a sort of temporary table that we can write SQL against

*Note: Creating a table in a subquery does **NOT** add it to the database - it disappears after the query returns its results!*

- You might have noticed that this is the same query we wrote earlier in the `CASE` statements section. We are using it as our subquery because we want the main query to use its results.
- Just like the other form of subquery, we wrap this in parentheses.
- The difference is that this subquery returns a table as a result and we give it an alias (`PriceRatingTable`), which is required.



**Now let's look at the main query:**

```sql
SELECT PriceRating, COUNT(*) AS TotalDishes
FROM -- SUBQUERY GOES HERE
GROUP BY PriceRating
ORDER BY TotalDishes DESC
```

Since we solved `step 1` - creating the pricing table - we can now move on to `step 2`, writing a query against it to calculate the number of dishes per pricing category. There's nothing new or fancy, here:

- The main query is just your typical aggregate query that we already know
- The only difference is that we are selecting data from a subquery by putting the subquery after `FROM` instead of another table

**Summary - Subqueries to answer multi-step questions**

- Subqueries can also be used to answer multi-step questions by creating custom tables that are needed to solve one of the steps
- They must be wrapped in parentheses
- The main query accesses the results in the subquery using the `FROM` clause
- By using subqueries like this, you can create queries that involve more than one data processing step to get your desired results

#### **Action Item - Using Subqueries**

The restaurant owners have been working on putting together a new menu and want to know the average price of a dish for each type of item. If you recall from earlier, we wrote a query to help them put that menu together using `UNION`. Here is the query to refresh your memory:

```sql
SELECT Name, Price, Type
FROM NewDishes
WHERE Type IN ('Appetizer', 'Dessert')
UNION
SELECT Name, Price, Type
FROM Dishes
WHERE Type IN ('Main', 'Beverage')
```

Find the average price of a dish for each `Type` using the query above as a subquery. Order your results by the average price in descending order.

`Expected Output:`

| Type      | MeanPrice |
|-----------|-----------|
| Main      | 9.094000  |
| Appetizer | 8.497500  |
| Dessert   | 7.250000  |
| Beverage  | 5.333333  |

<br/>
<details>
<summary>Click to reveal answer</summary>

```sql
SELECT Type, AVG(Price) AS MeanPrice
FROM (
    SELECT Name, Price, Type
    FROM NewDishes
    WHERE Type IN ('Appetizer', 'Dessert')
    UNION
    SELECT Name, Price, Type
    FROM Dishes
    WHERE Type IN ('Main', 'Beverage')
) AS NewMenu
GROUP BY Type
ORDER BY MeanPrice DESC
```

</details>

In [ ]:
%%sql



Congratulations, you've completed Lab 1!

---

## **Lab 2 - Common Table Expressions (CTE) and Subqueries**

While not technically a subquery, **Common Table Expressions (CTEs)** are a popular method in SQL for storing the results of intermediate queries to write more complex, multi-step queries.

In this section, you will learn:

- What a Common Table Expression (CTE) is
- Why they are useful
- How to rewrite a table subquery as a CTE for better readability

### **What is a CTE?**

A CTE, or Common Table Expression, is a temporary result set that you can reference within a `SELECT`, `INSERT`, `UPDATE`, or `DELETE` statement. CTEs are defined using the `WITH` clause. Once defined, a CTE is not stored as an object in the database; it exists just for the duration of the query. This means that any "tables" you create with your subquery will only be around when the query runs - once the query finishes, the table is no longer there.

### **How is a CTE Different from a Subquery?**

While CTEs and Subqueries are similar in that they let you write queries with multiple processing steps, they differ in some key ways:

- **Readability and Maintenance**: CTEs can make your queries more readable and easier to maintain. They allow you to name your temporary result set and reference it like you would a regular table. You can't do that with a subquery.

- **Reusability**: One of the main advantages of a CTE is that you can reference it multiple times in your main query, whereas a subquery would have to be rewritten each time you want to use it.

- **(Advanced Use Case) Recursive Queries**: CTEs can be recursive, meaning they can reference themselves. This is useful for hierarchical or tree-structured data, like organizational charts. Subqueries cannot do this. This is out of the scope of this course, but is worth mentioning regardless.

- **Ordering**: A CTE can be ordered using an ORDER BY clause, which can simplify the main query. In contrast, you cannot order a subquery that returns multiple rows (unless you're using it with an IN clause.)

### **Example - Rewriting our subquery using a CTE**

Subqueries are very useful, but they can be tricky to write and maintain simply because the syntax looks confusing. They require you to read queries from the inside-out to understand what's going on. That isn't the most intuitive thing to do.

This is a good reason to consider using CTEs instead. Using a CTE, we can make our query quite a bit more readable.

Let's rewrite our original subquery and use a CTE instead:

```sql
WITH pricing_scale AS ( -- We start each CTE using the WITH clause and naming our result. We're using "pricing_scale" here
    -- This is our "inner" query, still inside of parentheses
    SELECT Name,
          Price,
          Type,
          CASE
            WHEN Price <= 3.0 THEN 'Super Cheap'
            WHEN Price BETWEEN 4.0 AND 5.0 THEN 'Inexpensive'
            WHEN Price BETWEEN 5.0 AND 8.0 THEN 'Average'
            ELSE 'Pricey' -- Anything over 8.0 is Pricey
        END AS PriceRating
    FROM Dishes
)
-- This is the start of our "outer" query
SELECT PriceRating, COUNT(*) AS TotalDishes
FROM pricing_scale -- We reference our table using the name we gave it using the WITH clause
GROUP BY PriceRating
ORDER BY TotalDishes DESC
```

### **Breakdown - Rewriting our subquery as a CTE**

Our CTE, like the subquery, has two parts:
- The CTE itself, and
- A main query

You need both parts for the CTE to work correctly.



#### **The CTE**

We'll start with the CTE itself:

```sql
WITH pricing_scale AS (
    SELECT Name
        , Price
        , Type
        , CASE
            WHEN Price <= 3.0 THEN 'Super Cheap'
            WHEN Price BETWEEN 4.0 AND 5.0 THEN 'Inexpensive'
            WHEN Price BETWEEN 5.0 AND 8.0 THEN 'Average'
            ELSE 'Pricey' -- Anything over 8.0 is Pricey
        END AS PriceRating
    FROM Dishes
)
```

Let's break this down:

- First, we start the query using `WITH pricing_scale AS`, which is how we start every CTE query. This lets us give the CTE a descriptive name. It's similar to using an alias for a subquery.
- Just like in a subquery, we put our CTE query inside of a pair of parentheses. The difference here is that we're writing our main query underneath the CTE instead of on top of it. This can help a lot with readability.


#### **The Main Query of a CTE**

```sql
SELECT PriceRating, COUNT(*) AS TotalDishes
FROM pricing_scale -- We reference our table using the name we gave it using the WITH clause
GROUP BY PriceRating
ORDER BY TotalDishes DESC
```

Just like the subquery, we use a main query to perform additional processing on the result from the CTE query. The main difference, as mentioned above, is that the main query is underneath the CTE query rather than the other way around like in subqueries. Also, we can reference the name of the CTE query directly in the `FROM` clause, which we can't do with a subquery.


### **Action Item - Using a CTE**

For this exercise, you will simply rewrite the query you wrote for the subquery action item above as a CTE. Use the previous examples as a reference. The output will be the same as the previous query.

<br/>
<details>
<summary>Click here to reveal answer</summary>

```sql
WITH NewMenu AS (
    SELECT Name, Price, Type
    FROM NewDishes
    WHERE Type IN ('Appetizer', 'Dessert')
    UNION
    SELECT Name, Price, Type
    FROM Dishes
    WHERE Type IN ('Main', 'Beverage')
)
SELECT Type, AVG(Price) AS MeanPrice
FROM NewMenu
GROUP BY Type
ORDER BY MeanPrice DESC
```

</details>

In [ ]:
%%sql



---

### **<em>Comprehension Check</em> - Subqueries: Writing a query within a query**

Answer the questions below to check your understanding of what we have covered so far. Try to answer the questions first before looking at the answers:

*1. What is a subquery in SQL??*
<details>
    <summary>Click to reveal the answer</summary>
    <p>A subquery is a query nested inside another query.</p>
</details>


*2. What type of brackets are commonly used to encapsulate a subquery?*
<details>
    <summary>Click to reveal the answer</summary>
    <p>Parentheses () are commonly used to encapsulate a subquery.</p>
</details>


*3. What does CTE stand for?*
<details>
    <summary>Click to reveal the answer</summary>
    <p>`RIGHT JOIN` is the opposite of `LEFT JOIN`. It keeps all the records from the right-side table and only the matching records from the left-side table. Unmatched records from the left-side table will have `NULL` values.</p>
</details>

*4. What is the keyword used to start a CTE?*
<details>
    <summary>Click to reveal the answer</summary>
    <p>The keyword used to start a CTE is WITH.</p>
</details>

*5. Can a CTE be referenced more than once in a query?*
<details>
    <summary>Click to reveal the answer</summary>
    <p>Yes, a CTE can be referenced multiple times in a query.</p>
</details>

## Conclusion

Congratulations on completing SQL 102! You have a lot of new techniques to the skillset you began developing in SQL 101. Now you're ready to take on more complex questions. Great job! Let's keep moving forward to SQL 103, where you'll learn how to combine data from multiple tables.

As always, we encourage you to continue practicing and learning more. Use both the resources we provide and find opportunities to learn independently.

## More resources for further practice

- [SQL Bolt](https://sqlbolt.com/): The lessons here are a great introduction to SQL and you know the platform already!
- [Mode](https://mode.com/sql-tutorial/): A comprehensive SQL tutorial from beginner all the way to advanced SQL. There's even a data analytics with SQL tutorial. This is a great resource to learn about SQL in depth and practice what you learn in their online database.
- [StrataScratch](https://platform.stratascratch.com/coding): Practice coding questions geared toward data analysts and data scientists. You can solve coding problems used by real companies for technical interviews using PostgresSQL, Python, R, or MySQL. It's free to sign up!
- [Codecademy - Free Learn SQL Course](https://www.codecademy.com/learn/learn-sql): Codecademy is another great resource to learn SQL as well as most other languages. There are a lot of free resources here that can help you learn SQL, Python, R, and many other languages.
- [Socratica SQL (YouTube)](https://www.youtube.com/watch?v=nWyyDHhTxYU&list=PLih4ch-U2DiBbMoFK4ML9faT3k3MM2UQY): This is a great playlist that will get you started learning SQL with one of the most popular relational databases - Postgres.
- [DB Fiddle](https://dbfiddle.uk/): This site is like a SQL scratch pad. You can use it to practice doing stuff like creating tables and inserting data into them, and all sorts of other stuff that you might not be able to do so freely in a live database. It's a sandbox, basically. Here are a couple of links to fiddles with some data in them to play with: [fiddle 1](https://dbfiddle.uk/?rdbms=postgres_13&fiddle=366b683701596d3f7459b0411c15acd1) and [fiddle 2](https://dbfiddle.uk/?rdbms=postgres_13&fiddle=dfffc1939f629d9286c55d732fb656c5).


And don't forget to keep your [SQL Cheatsheet](https://martinmarroyo.github.io/sqlcheatsheetandresources-coop/) handy!